-understand semantic word vectors
-understand sentiment analysis
-leverage sentiment analysis for text classifition

word2vec is two-layer neutral network that processes text.Input is the text corpus, and output is set of vectors: feature vectors for words 

the purpose and usefulness of word2vec is to group the vectors of similar words together in vectorspace; that it detects similarities 

In [17]:
import spacy

In [18]:
nlp=spacy.load('en_core_web_md')

In [19]:
nlp(u'The quick fox jumped over').vector

array([-1.95381984e-01, -1.01039996e-02, -4.98833917e-02,  1.37300000e-01,
        5.62640000e-03,  6.10530488e-02, -1.38754010e-01,  9.18832049e-02,
        2.27846786e-01,  2.06998205e+00, -1.93725973e-01, -1.13400355e-01,
        3.59037928e-02, -1.77332342e-01, -2.51285017e-01,  3.52811925e-02,
       -3.72081921e-02,  7.71188021e-01, -1.22511007e-01, -2.68925995e-01,
       -9.66100022e-02,  1.20412391e-02,  3.31263952e-02, -2.42412418e-01,
        5.87682799e-02,  6.40620012e-03, -1.97531998e-01, -1.75930783e-01,
        1.16616013e-02, -2.15608403e-01, -1.06816545e-01,  3.20671976e-01,
        9.76475924e-02,  3.04087996e-02,  7.68330097e-02, -1.30164593e-01,
       -7.43820071e-02,  2.88162790e-02, -3.01937796e-02, -1.18265606e-01,
        1.02037407e-01,  2.27028038e-02, -3.59663442e-02, -1.62945986e-01,
        1.93638220e-01,  1.64349198e-01, -2.13824794e-01, -4.97430414e-02,
        1.21288195e-01,  4.31587957e-02, -1.67401195e-01,  6.79215938e-02,
       -6.88942000e-02, -

In [20]:
nlp(u'The quick fox jumped over').vector.shape  #300 dimension vector

(300,)

In [21]:
nlp(u'fox').vector.shape

(300,)

In [22]:
token=nlp(u'lion cat pet')

In [23]:
for token1 in token:
    for token2 in token:
        print(token1.text,token2.text,token1.similarity(token2))

lion lion 1.0
lion cat 0.5265437
lion pet 0.39923772
cat lion 0.5265437
cat cat 1.0
cat pet 0.7505456
pet lion 0.39923772
pet cat 0.7505456
pet pet 1.0


In [24]:
tokens=nlp(u'like love hate') 
for token1 in tokens:
    for token2 in tokens:
        print(token1.text,token2.text,token1.similarity(token2))
        
'''here it shows that although love and hate are the opposite meaning, 
but since they are often used in similar context(love a movie and hate a movie)'''

like like 1.0
like love 0.65790397
like hate 0.6574652
love like 0.65790397
love love 1.0
love hate 0.6393099
hate like 0.6574652
hate love 0.6393099
hate hate 1.0


'here it shows that although love and hate are the opposite meaning, \nbut since they are often used in similar context(love a movie and hate a movie)'

In [25]:
len(nlp.vocab.vectors) #there is essentially 20000 unique words in the vocabulary (large one has 684831)

20000

In [26]:
nlp.vocab.vectors.shape #this means that every word has 300 meanings, so the shape is 20000*300

(20000, 300)

In [27]:
# to check if the word has vector, or it is out of the vocabulary
tokens=nlp(u'dog cat nargle')
for token in tokens:
    print(token.text,token.has_vector,token.vector_norm,token.is_oov)

dog True 7.0336733 False
cat True 6.6808186 False
nargle False 0.0 True


### arithmatic of vectors
by adding two vectors to create a new vector

In [28]:
from scipy import spatial 

In [29]:
cosine_similarity=lambda vec1,vec2:1-spatial.distance.cosine(vec1,vec2)

In [30]:
king=nlp.vocab['king'].vector
man=nlp.vocab['man'].vector
woman=nlp.vocab['woman'].vector

In [31]:
# king-man+woman-----> new vector similar to Queen,princess and highness

In [32]:
new_vector=king-man+woman

In [33]:
compute_similarities=[ ]
#for all words in vocabulary
for word in nlp.vocab:
    if word.has_vector: #the word has vector
        if word.is_lower: #the word is lower case
            if word.is_alpha: #the word should not be number
                similarity=cosine_similarity(new_vector,word.vector)
                compute_similarities.append((word,similarity)) #append a tuple! so we use ()
            

In [34]:
computed_similarities=sorted(compute_similarities,key=lambda item:-item[1])  
#this is a fancy way of sorting, because item[1] is similarity, and - this is in the descending order

In [35]:
print([t[0].text for t in computed_similarities[:10]]) #print the first ten texts sorted by similarities (descending)

['king', 'queen', 'commoner', 'highness', 'prince', 'sultan', 'maharajas', 'princes', 'kumbia', 'kings']


# sentiment analysis
how to discern sentiment on raw unlabeled text ?
VADER(Valence Aware Dictionary for Sentiment Reasoning) is a model used for sentiment analysis that is useful for both polarity(positive/negative) and intensity(strength) of emotion.
it is in NLTK package, and can be directly used for unlabeld text

In [36]:
import nltk

In [37]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/gongchunyan/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [38]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [39]:
#create an instance of it, called std
sid=SentimentIntensityAnalyzer()

In [40]:
a='This is a good movie'

In [41]:
sid.polarity_scores(a)

{'neg': 0.0, 'neu': 0.508, 'pos': 0.492, 'compound': 0.4404}

In [42]:
a1='This was the best,most awesome movie EVER MADE!!!!'

In [43]:
sid.polarity_scores(a1)

{'neg': 0.0, 'neu': 0.571, 'pos': 0.429, 'compound': 0.7405}

In [44]:
a2='This was the worst movie i have ever seen!'

In [45]:
sid.polarity_scores(a2)

{'neg': 0.386, 'neu': 0.614, 'pos': 0.0, 'compound': -0.6588}

In [46]:
import pandas as pd

In [47]:
df=pd.read_csv('amazonreviews.tsv',sep='\t')

In [48]:
df.head()

,label,review
0,pos,Stuning even for the non-gamer: This sound tra...
1,pos,The best soundtrack ever to anything.: I'm rea...
2,pos,Amazing!: This soundtrack is my favorite music...
3,pos,Excellent Soundtrack: I truly like this soundt...
4,pos,"Remember, Pull Your Jaw Off The Floor After He..."


In [49]:
df['label'].value_counts()

neg    4971
pos    4780
Name: label, dtype: int64

In [50]:
df.dropna(inplace=True)

In [51]:
#check if there is any blanks in the reviews
blanks=[]
for i,lb,rv in df.itertuples():
    #index,label,review
    if type(rv)==str:
        if rv.isspace():
            blanks.append(i)

In [52]:
blanks

[]

In [53]:
#if there is any blanks,we drop those blanks
df.drop(blanks,inplace=True)

In [54]:
#check the polarity of the first review
df.iloc[0]['review']

'Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^'

In [55]:
sid.polarity_scores(df.iloc[0]['review'])

{'neg': 0.088, 'neu': 0.669, 'pos': 0.243, 'compound': 0.9454}

In [60]:
df['scores']=df['review'].apply(lambda review:sid.polarity_scores(review)) #get the score for every single review

In [61]:
df.head()

,label,review,scores,compound
0,pos,Stuning even for the non-gamer: This sound tra...,"{'neg': 0.088, 'neu': 0.669, 'pos': 0.243, 'co...",0.0
1,pos,The best soundtrack ever to anything.: I'm rea...,"{'neg': 0.018, 'neu': 0.837, 'pos': 0.145, 'co...",0.0
2,pos,Amazing!: This soundtrack is my favorite music...,"{'neg': 0.04, 'neu': 0.692, 'pos': 0.268, 'com...",0.0
3,pos,Excellent Soundtrack: I truly like this soundt...,"{'neg': 0.09, 'neu': 0.615, 'pos': 0.295, 'com...",0.0
4,pos,"Remember, Pull Your Jaw Off The Floor After He...","{'neg': 0.0, 'neu': 0.746, 'pos': 0.254, 'comp...",0.0


In [62]:
df['compound']=df['scores'].apply(lambda d:d['compound'])

In [63]:
df.head()

,label,review,scores,compound
0,pos,Stuning even for the non-gamer: This sound tra...,"{'neg': 0.088, 'neu': 0.669, 'pos': 0.243, 'co...",0.9454
1,pos,The best soundtrack ever to anything.: I'm rea...,"{'neg': 0.018, 'neu': 0.837, 'pos': 0.145, 'co...",0.8957
2,pos,Amazing!: This soundtrack is my favorite music...,"{'neg': 0.04, 'neu': 0.692, 'pos': 0.268, 'com...",0.9858
3,pos,Excellent Soundtrack: I truly like this soundt...,"{'neg': 0.09, 'neu': 0.615, 'pos': 0.295, 'com...",0.9814
4,pos,"Remember, Pull Your Jaw Off The Floor After He...","{'neg': 0.0, 'neu': 0.746, 'pos': 0.254, 'comp...",0.9781


In [64]:
df['compound_score']=df['compound'].apply(lambda score:'pos' if score>=0 else 'neg')

In [65]:
df.head()

,label,review,scores,compound,compound_score
0,pos,Stuning even for the non-gamer: This sound tra...,"{'neg': 0.088, 'neu': 0.669, 'pos': 0.243, 'co...",0.9454,pos
1,pos,The best soundtrack ever to anything.: I'm rea...,"{'neg': 0.018, 'neu': 0.837, 'pos': 0.145, 'co...",0.8957,pos
2,pos,Amazing!: This soundtrack is my favorite music...,"{'neg': 0.04, 'neu': 0.692, 'pos': 0.268, 'com...",0.9858,pos
3,pos,Excellent Soundtrack: I truly like this soundt...,"{'neg': 0.09, 'neu': 0.615, 'pos': 0.295, 'com...",0.9814,pos
4,pos,"Remember, Pull Your Jaw Off The Floor After He...","{'neg': 0.0, 'neu': 0.746, 'pos': 0.254, 'comp...",0.9781,pos


In [1]:
from sklearn.metrics import accuracy_score,classification_report

In [70]:
accuracy_score(df['label'],df['compound_score'])

0.7091580350733259

In [71]:
print(classification_report(df['label'],df['compound_score']))

              precision    recall  f1-score   support

         neg       0.86      0.51      0.64      4971
         pos       0.64      0.91      0.75      4780

    accuracy                           0.71      9751
   macro avg       0.75      0.71      0.70      9751
weighted avg       0.75      0.71      0.70      9751

